In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sys
sys.path.append('../../../../infrastructure/tools')
from utilities import *
from plotting import *

In [2]:
raw_user_data = pd.read_csv("../../../../fraud_detection_transac/data/users_data.csv").sort_values('id').reset_index(drop=True)

In [3]:
raw_user_data.shape

(2000, 14)

In [4]:
raw_user_data = raw_user_data.rename(columns={'id':'user_id'})

In [5]:
raw_user_data.head()

,user_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,0,33,69,1986,3,Male,858 Plum Avenue,43.59,-70.33,$29237,$59613,$36199,763,4
1,1,43,74,1976,4,Female,113 Burns Lane,30.44,-87.18,$22247,$45360,$14587,704,3
2,2,48,64,1971,8,Male,6035 Forest Avenue,40.84,-73.87,$13461,$27447,$80850,673,5
3,3,49,65,1970,12,Male,840 Elm Avenue,33.89,-98.51,$13705,$27943,$18693,681,4
4,4,54,72,1965,3,Female,6016 Little Creek Boulevard,47.61,-122.30,$37485,$76431,$115362,716,5


In [6]:
raw_transaction_data = pd.read_csv("../../../../fraud_detection_transac/data/transactions_data.csv").sort_values(['id','client_id']).reset_index(drop=True)

In [7]:
raw_transaction_data.shape

(13305915, 12)

In [8]:
raw_transaction_data = raw_transaction_data.rename(columns={'id':'transaction_id','client_id':'user_id'})

In [9]:
raw_transaction_data.head()

,transaction_id,date,user_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [10]:
raw_card_data = pd.read_csv("../../../../fraud_detection_transac/data/cards_data.csv").sort_values(['id','client_id']).reset_index(drop=True)

In [11]:
raw_card_data.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,0,1362,Amex,Credit,393314135668401,04/2024,866,YES,2,$33900,01/1991,2014,No
1,1,550,Mastercard,Credit,5278231764792292,06/2024,396,YES,1,$11600,01/1994,2013,No
2,2,556,Mastercard,Debit,5889825928297675,09/2021,422,YES,1,$19948,01/1995,2011,No
3,3,1937,Visa,Credit,4289888672554714,04/2020,736,YES,2,$16400,01/1995,2015,No
4,4,1981,Mastercard,Debit,5433366978583845,03/2024,530,YES,2,$19439,01/1997,2007,No


In [12]:
raw_card_data = raw_card_data.rename(columns={'id':'card_id','client_id':'user_id'})

In [13]:
labels = pd.read_json("../../../../fraud_detection_transac/data/train_fraud_labels.json").reset_index()

In [14]:
labels = labels.rename(columns={'index':'transaction_id'})

In [15]:
labels.shape

(8914963, 2)

In [16]:
mcc = pd.read_json("../../../../fraud_detection_transac/data/mcc_codes.json",typ='series').reset_index()

In [17]:
mcc = mcc.rename(columns={'index':'mcc', 0:'merchant_type'})

In [18]:
transactions = raw_transaction_data.merge(mcc,on='mcc', how='left').merge(labels,on='transaction_id',how='left')

In [19]:
transactions = transactions.sort_values(by=['mcc','user_id','transaction_id']).reset_index(drop=True)

In [20]:
transactions.shape

(13305915, 14)

In [21]:
raw_card_data.head()

,card_id,user_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,0,1362,Amex,Credit,393314135668401,04/2024,866,YES,2,$33900,01/1991,2014,No
1,1,550,Mastercard,Credit,5278231764792292,06/2024,396,YES,1,$11600,01/1994,2013,No
2,2,556,Mastercard,Debit,5889825928297675,09/2021,422,YES,1,$19948,01/1995,2011,No
3,3,1937,Visa,Credit,4289888672554714,04/2020,736,YES,2,$16400,01/1995,2015,No
4,4,1981,Mastercard,Debit,5433366978583845,03/2024,530,YES,2,$19439,01/1997,2007,No


In [22]:
cards = raw_card_data.merge(raw_user_data, on=['user_id'], how='left')

In [23]:
cards = cards.sort_values(by=['user_id','card_id']).reset_index(drop=True)

In [24]:
cards.shape

(6146, 26)

In [25]:
transactions.to_csv("../../../data/intermediate/transactions.csv", index=False)

In [26]:
cards.to_csv("../../../data/intermediate/cards.csv", index=False)